### 2.5. Advanced Geospatial Plotting

In [2]:
# Import Libraries

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Import Data

df = pd.read_csv('nycbike_clean.csv', index_col = 0)

/var/folders/8k/dxgtncgd49g2ryx410bsc12r0000gn/T/ipykernel_38427/425896445.py:3: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nycbike_clean.csv', index_col = 0)


In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,trips_per_day,trip_duration
0,F2147CAB2E003A12,classic_bike,2022-01-01 00:56:36,2022-01-01 01:06:55,Cedar St & Evergreen Ave,4721.01,Willoughby Ave & Tompkins Ave,4665.02,40.696710,-73.928070,40.694254,-73.946269,member,2022-01-01,11.6,20895,10.316667
1,27FF7FCE0F8A02DF,classic_bike,2022-01-01 15:42:26,2022-01-01 16:02:59,Albany St & Greenwich St,5145.02,Bayard St & Baxter St,5351.03,40.709267,-74.013247,40.716021,-73.999744,casual,2022-01-01,11.6,20895,20.550000
2,E413BC1D0B4C6031,classic_bike,2022-01-01 15:37:55,2022-01-01 16:03:05,Albany St & Greenwich St,5145.02,Bayard St & Baxter St,5351.03,40.709267,-74.013247,40.716021,-73.999744,casual,2022-01-01,11.6,20895,25.166667
3,1967ADD90811CE78,electric_bike,2022-01-01 17:35:23,2022-01-01 17:46:05,Cabrini Blvd & W 177 St,8358.1,Broadway & W 220 St,8715.01,40.848282,-73.941343,40.872412,-73.912807,member,2022-01-01,11.6,20895,10.700000
4,D199C729356BA836,classic_bike,2022-01-01 19:25:20,2022-01-01 19:41:53,5 Ave & E 72 St,7100.07,W 41 St & 8 Ave,6602.03,40.772828,-73.966853,40.756405,-73.990026,casual,2022-01-01,11.6,20895,16.550000


#### Data Preprocessing

In [5]:
# Create a value column and group by start and end station 

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])['value'].count().reset_index()

In [6]:
df_group

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,value
0,1 Ave & E 110 St,1 Ave & E 110 St,40.792062,-73.937756,40.792327,-73.938300,1
1,1 Ave & E 110 St,1 Ave & E 110 St,40.792078,-73.937701,40.792327,-73.938300,1
2,1 Ave & E 110 St,1 Ave & E 110 St,40.792142,-73.937926,40.792327,-73.938300,1
3,1 Ave & E 110 St,1 Ave & E 110 St,40.792144,-73.937845,40.792327,-73.938300,1
4,1 Ave & E 110 St,1 Ave & E 110 St,40.792175,-73.937893,40.792327,-73.938300,1
...,...,...,...,...,...,...,...
5089672,Yankee Ferry Terminal,Yankee Ferry Terminal,40.687181,-74.016553,40.687066,-74.016756,1
5089673,Yankee Ferry Terminal,Yankee Ferry Terminal,40.687197,-74.016486,40.687066,-74.016756,1
5089674,Yankee Ferry Terminal,Yankee Ferry Terminal,40.687216,-74.016586,40.687066,-74.016756,1
5089675,Yankee Ferry Terminal,Yankee Ferry Terminal,40.687234,-74.016551,40.687066,-74.016756,1


In [8]:
#Checking the grouping

print(df_group['value'].sum())
print(df.shape)

30458353
(30458353, 18)


In [9]:
df_group['value'].describe()

count    5.089677e+06
mean     5.984339e+00
std      4.507411e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.217800e+04
Name: value, dtype: float64

In [12]:
df_group['value'].value_counts()

value
1       4404611
2        129911
3         73704
4         50550
5         37692
         ...   
2061          1
2354          1
1808          1
1564          1
4754          1
Name: count, Length: 1913, dtype: int64

In [13]:
df_group.to_csv('df_groupby_station.csv')

In [25]:
# Create a dataframe with most popular routes

df_popular = df_group.nlargest(1000, 'value')

In [26]:
df_popular.to_csv('df_groupby_station_popular.csv')

In [32]:
df_popular.shape

(1000, 7)

#### Initialising the Map

### Widget is never loading  so i try again with below method

In [28]:
# Create KeplerGl instance

map = KeplerGl()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [29]:
map.add_data(data=df_popular, name='NY_CitiBike')

In [30]:
map

/opt/anaconda3/envs/venv_nycitybike/lib/python3.12/site-packages/IPython/core/formatters.py:370: FormatterWarning: text/html formatter returned invalid type <class 'bytes'> (expected <class 'str'>) for object: KeplerGl(data={'NY_CitiBike': {'index': [1407487, 639170, 3776467, 2734441, 1241064, 4990156, 508380, 149225, 3862611, 4337161, 1441277, 42358, 1401028, 3549099, 3809462, 1433718, 522048, 5089607, 1196844, 4954934, 4998862, 3521890, 1459251, 52418, 3532812, 4986190, 124061, 2068381, 4345607, 3613304, 2913735, 4358367, 3862798, 5089501, 4345191, 88522, 3776063, 3717821, 4087700, 3432614, 1506800, 3485394, 2561334, 4358157, 515930, 52981, 4680309, 528968, 2356042, 1452686, 156767, 594718, 1155584, 3231258, 71268, 75797, 4134253, 1972296, 861114, 4980391, 4420388, 582215, 3323727, 1405752, 3920213, 3523725, 1078116, 4575698, 1575404, 94761, 1539651, 4971066, 3611714, 1279094, 4566728, 4357810, 1751958, 148846, 132616, 70659, 3379741, 4127190, 3800559, 1420085, 4918522, 4998722, 1035

KeplerGl(data={'NY_CitiBike': {'index': [1407487, 639170, 3776467, 2734441, 1241064, 4990156, 508380, 149225, …

In [33]:
map.save_to_html(file_name='NY_CitiBike_popular.html')

Map saved to NY_CitiBike_popular.html!


##### Based on the map, we can see that a lot of customers from CitiBike are using their services around the central park areas and along road near the Hudson River.